In [ ]:
import pandas as pd
from transformers import pipeline
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

file_path = './imdb_reviews_unique.csv'
reviews_df = pd.read_csv(file_path)

sentiment_model = pipeline("sentiment-analysis", truncation=True)

def analyze_sentiment(title, review):
    full_text = title + ". " + review
    sentences = sent_tokenize(full_text)
    scores = []

    for sentence in sentences:
        max_length = 512
        parts = [sentence[i:i + max_length] for i in range(0, len(sentence), max_length)]

        for part in parts:
            if part.strip() != '':
                result = sentiment_model(part)[0]
                scores.append(result['score'] if result['label'] == 'POSITIVE' else 1 - result['score'])

    return sum(scores) / len(scores) if scores else 0

reviews_df['Sentiment_Score'] = reviews_df.apply(lambda x: analyze_sentiment(x['Title'], x['Review']), axis=1)

reviews_df.head()

output_path = "../data/Sentiment_Score.csv"
reviews_df.to_csv(output_path, index=False)